In [3]:
# Now lets download the trained model from the last notebook using joblib
import joblib

# Load the trained model from last notebook
model = joblib.load('/content/drive/MyDrive/Colab Notebooks/XAIentists/Trained Models/xgboost_model.pkl')

In [35]:
import shap
import lime
import lime.lime_tabular
import joblib
import numpy as np
import pandas as pd
import xgboost
import xgboost
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder


In [11]:
data =pd.read_csv('/content/drive/MyDrive/Colab Notebooks/XAIentists/Data/Processed/compas-scores-two-years-processed.csv')

# Drop the two target columns
X = data.drop('two_year_recid', axis=1)
y = data['two_year_recid']

#model = joblib.load('../Trained Models/xgboost_model.pkl')

model = joblib.load("/content/drive/MyDrive/Colab Notebooks/XAIentists/Trained Models/xgboost_model.pkl", mmap_mode="r")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
#rf_model.fit(X_train, y_train)

feature_names = X_train.columns.tolist()

In [ ]:
print(X_test.shape)  # Ensure it has 19 features as in the training set
print(X_train.shape) # Ensure it has 19 features as well
print(instance.shape) # This should return (19,) for 19 features

In [62]:
encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_train_encoded = encoder.fit_transform(X_train)

# Ensure it's a NumPy array
X_train_encoded = pd.DataFrame(X_train_encoded)

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_encoded.values,
    training_labels=y_train,
    feature_names=encoder.get_feature_names_out(),
    class_names=["No Recidivism", "Recidivism"],
    mode="classification"
)
# Select a sample instance from the test set
i = 5  # Choose any index
instance = X_test.iloc[i].values
X_test = X_test[X_train.columns]  # Ensure X_test has the same columns as X_train

#instance = X_train.iloc[0]  # Ensure it's a valid row from your training data

# Generate explanation
exp = explainer.explain_instance(instance, model.predict_proba, num_features=10)

# Visualize explanation
exp.show_in_notebook()

In [ ]:
print(X_train.shape)  # Check the shape of the dataset

dmatrix = xgb.DMatrix(X_train, enable_categorical=True)
print(dmatrix.num_row())  # Verify the number of rows in the DMatrix
print(dmatrix.num_col())

In [56]:
# For numeric columns, you can fill missing values with the mean or median
#X_train['r_days_from_arrest'].fillna(X_train['r_days_from_arrest'].mean(), inplace=True)
X_train['r_days_from_arrest'] = X_train['r_days_from_arrest'].fillna(X_train['r_days_from_arrest'].mean())
#X_train['r_charge_desc'].fillna(X_train['r_charge_desc'].mean(), inplace=True)
X_train['r_charge_desc'] = X_train['r_charge_desc'].fillna(X_train['r_charge_desc'].mean())
#X_train['vr_charge_desc'].fillna(X_train['vr_charge_desc'].mean(), inplace=True)
X_train['vr_charge_desc'] = X_train['vr_charge_desc'].fillna(X_train['vr_charge_desc'].mean())
#X_train['r_length_of_stay'].fillna(X_train['r_length_of_stay'].mean(), inplace=True)
X_train['r_length_of_stay'] = X_train['r_length_of_stay'].fillna(X_train['r_length_of_stay'].mean())

In [57]:
# Drop rows with any missing values
X_train.dropna(inplace=True)

In [ ]:
print(X_train.isna().sum())  # Check for missing values in the dataset

print(X_train.shape)  # Check if the number of rows and columns have changed

In [ ]:
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

label_encoder = LabelEncoder()
X_train['sex'] = label_encoder.fit_transform(X_train['sex'])
X_train['age_cat'] = label_encoder.fit_transform(X_train['age_cat'])
X_train['race'] = label_encoder.fit_transform(X_train['race'])
X_train['c_charge_degree'] = label_encoder.fit_transform(X_train['c_charge_degree'])
X_train['r_charge_degree'] = label_encoder.fit_transform(X_train['r_charge_degree'])
X_train['vr_charge_degree'] = label_encoder.fit_transform(X_train['vr_charge_degree'])

dmatrix = xgb.DMatrix(X_train, enable_categorical=True)

# Create SHAP explainer
explainer = shap.TreeExplainer(model)  # Random Forest is tree-based

X_train['sex'] = X_train['sex'].astype('category')

# Compute SHAP values
shap_values = explainer.shap_values(X_train)
#shap_values = explainer.shap_values(dmatrix)

# Summary plot (global feature importance)
shap.summary_plot(shap_values[1], X_test, feature_names=feature_names)  # Class 1 (Recidivism)

# Explanation for a single instance
shap.force_plot(explainer.expected_value[1], shap_values[1][i], X_test.iloc[i], matplotlib=True)

# Initialize JavaScript visualization to understand how the model makes decisions (for each feature)
shap.initjs()
shap.force_plot(shap_values[1][0], X_test[0], feature_names=feature_names)

In [ ]:
# Check the data types of all columns
print(X_train.dtypes)